In [38]:
using Distributions
using LightGraphs, SimpleWeightedGraphs
using GraphPlot
using LightGraphs: smallgraph
using Flux
using LinearAlgebra
using ProximalOperators

In [39]:
f1(x) = 1/(1+exp(x^2))
function edge_probability(f1, i::Int, j::Int, coordinates) 
    yi, yj = coordinates[[i,j], :]
    return f1(norm(yi - yj))
end

edge_probability (generic function with 1 method)

In [40]:
function biased_sample(n, k, p)
    """
    n is the number of samples to draw
    k is the number of catergories, here is just the number of nodes
    p is the corresponding probability of k, everything in p sums to 1
    """
    r = 1
    a = Vector{Int}()
    for i in 1:k
        q = rand(Binomial(n,p[i]/r))
        x = fill(i,q)
        append!(a, x)
        n = n - q
        r = r - p[i]
    end
    return a
end

biased_sample (generic function with 1 method)

In [41]:
function likelihood(coordinates)  
    g =  smallgraph(:karate)
    A = adjacency_matrix(g)
    A =  Matrix(A)
    B = -1*(A.-1)
    g2 = Graph(B) 
    
    nodes = Vector{Int}()
    for n in vertices(g)
       append!(nodes,n) 
    end
    
    
    negsource = Vector{Int}()
    negdestin = Vector{Int}()
    for e in edges(g2)
        u, v = src(e), dst(e)
        append!(negsource, u)
        append!(negdestin, v)
    end
    
    weight2 = fill(1,ne(g2))
    weight2_norm = weight2./sum(weight2)
    
    
    num_negative_edges::Int=10
    γ = 0.5
    lik = 0

    for ed in edges(g)
        ei, ej = src(ed), dst(ed)
        indi = findall(x->x==ei, nodes)[1]
        indj = findall(x->x==ej, nodes)[1]
        lik += log(edge_probability(f1, indi, indj, coordinates))
        
        neg_ind = findall(x->x==ej,negdestin)

        num_of_neg_ej = length(neg_ind)
        neg_ind_weight = weight2_norm[neg_ind]
        neg_ind_weight_norm = neg_ind_weight./(sum(neg_ind_weight)+0.000001)
        
        for i in 1:num_negative_edges
            e = biased_sample(1, num_of_neg_ej,neg_ind_weight_norm)
            edge_ind = neg_ind[e]
            
            indii = findall(x->x==negsource[edge_ind][1], nodes)[1]
            indjj = findall(x->x==negdestin[edge_ind][1], nodes)[1]
            lik += γ * log(1 - edge_probability(f1, indii, indjj, coordinates))
        end
    
    end
    return -lik
end

likelihood (generic function with 1 method)

In [42]:
g =  smallgraph(:karate) 

Y = rand(nv(g), 2) # I think julia is faster columnwise. Not sure

likelihood(Y)

305.7637841321173

In [37]:
gradient(likelihood, Y)

LoadError: UndefVarError: gradient not defined

LoadError: invalid redefinition of constant GD_setting